# [1757. Recyclable and Low Fat Products](https://leetcode.com/problems/recyclable-and-low-fat-products/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Products

<pre>
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_id  | int     |
| low_fats    | enum    |
| recyclable  | enum    |
+-------------+---------+</pre>

product_id is the primary key (column with unique values) for this table.
low_fats is an ENUM (category) of type ('Y', 'N') where 'Y' means this product is low fat and 'N' means it is not.
recyclable is an ENUM (category) of types ('Y', 'N') where 'Y' means this product is recyclable and 'N' means it is not.
 

Write a solution to find the ids of products that are both low fat and recyclable.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Products table:
<pre>
+-------------+----------+------------+
| product_id  | low_fats | recyclable |
+-------------+----------+------------+
| 0           | Y        | N          |
| 1           | Y        | Y          |
| 2           | N        | Y          |
| 3           | Y        | Y          |
| 4           | N        | N          |
+-------------+----------+------------+
</pre>
Output:
<pre> 
+-------------+
| product_id  |
+-------------+
| 1           |
| 3           |
+-------------+
</pre>
Explanation: Only products 1 and 3 are both low fat and recyclable.

In [1]:
# Pandas Schema

import pandas as pd

data = [['0', 'Y', 'N'], ['1', 'Y', 'Y'], ['2', 'N', 'Y'], ['3', 'Y', 'Y'], ['4', 'N', 'N']]
products = pd.DataFrame(data, columns=['product_id', 'low_fats', 'recyclable']).astype(
    {'product_id': 'int64', 'low_fats': 'category', 'recyclable': 'category'})

In [2]:
import warnings

#converting to spark dataframe
warnings.simplefilter('ignore')
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

products_df = spark.createDataFrame(products)
products_df.printSchema()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 10:39:06 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 10:39:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 10:39:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- product_id: long (nullable = true)
 |-- low_fats: string (nullable = true)
 |-- recyclable: string (nullable = true)


In [3]:
products_df.show(truncate=False)

+----------+--------+----------+
|product_id|low_fats|recyclable|
+----------+--------+----------+
|0         |Y       |N         |
|1         |Y       |Y         |
|2         |N       |Y         |
|3         |Y       |Y         |
|4         |N       |N         |
+----------+--------+----------+


In [4]:
# in Spark Dataframe
import pyspark.sql.functions as F

products_df.filter((F.col('low_fats') == 'Y') & (F.col('recyclable') == 'Y')).select('product_id').show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+


In [5]:
# in Spark SQL

products_df.createOrReplaceTempView('Products')
spark.sql('''
select product_id from products where low_fats="Y" and recyclable="Y";
''').show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+


In [6]:
spark.stop()